In [1]:
%matplotlib inline
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

import sys, os
# sys.path.append('../largescale_smc/')
# sys.path.append('../third_party')

In [4]:
import numpy as np   
import pandas as pd
import pdb
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from tqdm import tqdm
import sys

from time import time
from largescale_smc.largescale_missingness import *
from largescale_smc.largescale_utils import *
from largescale_smc.largescale_method import *

In [5]:
from surprise import SVD
from surprise import accuracy

In [6]:
%%memit
start_time = time()
#file_path = os.path.expanduser("../data/ml-100k/u.data")
file_path = os.path.expanduser("../data/ml-10m/ratings_no_timestamp.dat")
reader = DataReader(line_format="user item rating", sep="::")
splitter = DataSplitter(file_path, reader)
train_samples, calib_samples, missing_idxs = splitter.sample_train_calib(k=5)
print(f"Elapsed time: {time()-start_time} seconds")

10000054it [00:13, 726956.71it/s]

Loaded 10000054 ratings from 69878 users and 10677 items.



100%|███████████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 749.93it/s]


Elapsed time: 29.965102195739746 seconds
peak memory: 2576.08 MiB, increment: 2401.69 MiB


In [7]:
%%memit
random_state=0
start_time = time()
trainset = construct_trainset(train_samples, reader.rating_scale)
algo = SVD(n_factors=30, n_epochs=10, biased=False, verbose=True)
algo.fit(trainset)
print(f"Fitting time: {time()-start_time} seconds")

# start_time = time()
# # Then compute RMSE
# predictions = algo.test(raw_testset)
# accuracy.rmse(predictions)
# print(f"Testing time: {time()-start_time} seconds")

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Fitting time: 82.93631529808044 seconds
peak memory: 4064.29 MiB, increment: 1574.48 MiB


In [8]:
%%memit
start_time = time()
w_obs=LogisticMFProbs(random_state=random_state)
w_obs.fit(train_samples+calib_samples)
print(f"Elapsed time: {time()-start_time} seconds")

100%|████████████████████████████████████████████████████████████████████████████| 30/30 [00:21<00:00,  1.38it/s]


Elapsed time: 55.21460318565369 seconds
peak memory: 4691.58 MiB, increment: 779.19 MiB


In [9]:
%%memit
start_time = time()
k=5
Mhat = create_batch_rating_predictor(algo)
ci_method = SimulCI_ls(splitter.config, calib_samples, k, Mhat, w_obs)
print(f"Elapsed time: {time()-start_time} seconds")

Elapsed time: 0.04078555107116699 seconds
peak memory: 3932.42 MiB, increment: 0.20 MiB


In [10]:
calib_idxs = [(u, i) for u,i,r in calib_samples]
calib_ratings = [r for u,i,r in calib_samples]

In [ ]:
%%memit
start_time = time()
alpha = 0.1
allow_inf= False
df = ci_method.get_CI(calib_idxs[:1], alpha, allow_inf=allow_inf)
print(f"Elapsed time: {time()-start_time} seconds")

Computing conformal prediction intervals for 0 test queries...
> c:\users\liang\documents\github\conformal-matrix-completion\largescale_smc\largescale_method.py(361)_compute_universal()
    359         pdb.set_trace()
    360 
--> 361         for key in self.n_obs.keys():
    362             obs_entries = set([entry for entry, rating in self.config.key_to_samples[key]])
    363             all_entries = self.config.user_ids if self.key_dim else self.config.item_ids



ipdb>  l


    356         delta = 0.0      # Sum of sampling weights on the missing set
    357         all_observed_weights = []
    358 
    359         pdb.set_trace()
    360 
--> 361         for key in self.n_obs.keys():
    362             obs_entries = set([entry for entry, rating in self.config.key_to_samples[key]])
    363             all_entries = self.config.user_ids if self.key_dim else self.config.item_ids
    364             miss_entries = all_entries - obs_entries
    365 
    366             # Update the quantities per key



ipdb>  s


> c:\users\liang\documents\github\conformal-matrix-completion\largescale_smc\largescale_method.py(362)_compute_universal()
    360 
    361         for key in self.n_obs.keys():
--> 362             obs_entries = set([entry for entry, rating in self.config.key_to_samples[key]])
    363             all_entries = self.config.user_ids if self.key_dim else self.config.item_ids
    364             miss_entries = all_entries - obs_entries



...

*** NameError: name 'w_obs' is not defined


ipdb>  self.w_obs[(key,obs_entries[0])]


*** TypeError: 'set' object is not subscriptable


ipdb>  obs_entries


{'10269', '12782', '5288', '66445', '19431', '12189', '59269', '55942', '28371', '57689', '57759', '62476', '775', '5822', '30112', '58108', '53460', '42696', '58135', '51033', '11181', '58516', '5039', '14649', '6534', '24002', '70347', '22742', '24695', '3149', '17634', '8974', '5290', '3949', '38258', '47153', '43451', '23450', '25671', '25802', '64756', '8628', '57254', '51897', '15794', '3542', '51880', '11890', '62062', '2596', '11175', '41038', '31321', '65176', '65031', '67123', '44190', '45997', '48709', '13012', '24100', '28479', '54603', '3067', '24812', '68426', '5955', '38139', '63317', '23902', '38928', '10978', '43561', '22332', '68081', '43174', '57276', '31758', '61023', '70903', '51528', '57934', '26387', '34786', '51985', '62615', '58049', '6748', '41168', '48723', '9827', '20668', '33243', '48429', '40359', '21182', '51205', '21679', '62489', '63394', '69266', '45259', '34287', '21199', '30780', '59731', '26893', '43646', '33517', '41440', '40075', '41827', '36801',

In [ ]:
plt.hist(w_obs.mean_user_factor)

In [ ]:
plt.hist(w_obs.user_factors[1])

In [ ]:
w_obs.user_factors[1]

In [ ]:
w_obs.mean_user_factor

In [ ]:
alph = ['a', 'b', 'c']

In [ ]:
num = [1,2,3]

In [ ]:
tt=list(zip(alph, num))

In [ ]:
tt[0][1]

In [ ]:
np.abs([1,2,4]-[0,1,1])

In [ ]:
import sys

In [ ]:
sys.float_info.max

In [ ]:
ss=set([1,23,4,5,1])

In [ ]:
np.prod(np.array([1,23,4]))

In [ ]:
test= 1

In [ ]:
test /= 1+1

In [ ]:
test

In [ ]:
np.array([1,0,0]) | np.array([1,0,1])